In [1]:
from pymilvus import Collection, connections

connections.connect(alias="default", host="localhost", port="19530")
c = Collection("documentos_legales_v2")
print("Número de entidades:", c.num_entities)

Número de entidades: 0


In [3]:
# # Consulta los primeros 5 registros y muestra el campo metadata
# res = c.query(expr=None, output_fields=["metadata"], limit=5)
# for r in res:
#     print(r)

In [5]:
from pymilvus import Collection, connections, FieldSchema, CollectionSchema, DataType, utility

connections.connect(alias="default", host="localhost", port="19530")
collection_name = "documentos_legales_v1"

if collection_name not in utility.list_collections():
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
        FieldSchema(name="metadata", dtype=DataType.JSON)
    ]
    schema = CollectionSchema(fields, description="Test")
    c = Collection(name=collection_name, schema=schema)
else:
    c = Collection(collection_name)

# Inserta un registro de prueba
import numpy as np
dummy_embedding = np.random.rand(384).tolist()
dummy_metadata = {"test": "ok", "foo": 123}
c.insert([[dummy_embedding], [dummy_metadata]])
print("Ahora hay:", c.num_entities, "entidades")

Ahora hay: 0 entidades


In [6]:
c.insert([
    [dummy_embedding],  # embedding
    [dummy_metadata]    # metadata
])

(insert count: 1, delete count: 0, upsert count: 0, timestamp: 458957478936444929, success count: 1, err count: 0

In [7]:
from pymilvus import Collection, connections, FieldSchema, CollectionSchema, DataType, utility

connections.connect(alias="default", host="localhost", port="19530")
collection_name = "documentos_legales_v2"

if collection_name not in utility.list_collections():
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
        FieldSchema(name="metadata", dtype=DataType.JSON)
    ]
    schema = CollectionSchema(fields, description="Test")
    c = Collection(name=collection_name, schema=schema)
else:
    c = Collection(collection_name)

import numpy as np
dummy_embedding = np.random.rand(384).tolist()
dummy_metadata = {"test": "ok", "foo": 123}

# El orden debe ser [embedding, metadata] porque id es auto_id
c.insert([
    [dummy_embedding],  # embedding
    [dummy_metadata]    # metadata
])
c.flush()
print("Ahora hay:", c.num_entities, "entidades")

Ahora hay: 3 entidades


In [9]:
from pymilvus import connections, Collection

connections.connect("default", host="localhost", port="19530")
collection = Collection("documentos_legales_v2")
print("Número de entidades:", collection.num_entities)
results = collection.query(expr="id > 0", output_fields=["*"])
print(results)


Número de entidades: 0
data: ['{\'embedding\': [-0.046585135, 0.027574357, -0.03498315, -0.10311836, 0.011605326, 0.030422999, 0.0003140924, 0.08414453, -0.030142967, 0.12352099, 0.045272782, -0.05346095, 0.0064016995, 0.026048493, 0.097355545, 0.018193899, -0.04329276, -0.01140756, -0.019126974, 0.09105185, 0.15103193, 0.06415071, -0.086258575, 0.06979194, -0.07092241, -0.087243356, -0.029582843, -0.014069104, -0.062322676, -0.0260604, 0.061108753, 0.036916103, 0.099732466, -0.0077993167, 0.038294338, -0.05456235, -0.031549323, -0.017052626, 0.009323528, 0.03601632, -0.07757551, -0.0141908545, -0.07065223, -0.025707131, -0.053608812, -0.038139325, 0.04254515, 0.094629385, -0.044809874, -0.051466815, -0.022018146, -0.03465388, -0.012488431, -0.019610645, -0.064117566, -0.016599478, -0.021933006, -0.07089551, 0.062493224, 0.021384297, 0.014294335, 0.053591393, -0.040102206, -0.033624716, 0.027430259, 0.010725921, 0.07822209, -0.050778177, -0.06733895, 0.08100739, 0.09119888, -0.05549346

In [10]:
from pymilvus import connections, Collection
import json

# Conectar a Milvus
connections.connect("default", host="localhost", port="19530")
collection = Collection("documentos_legales_v2")

print("=== VERIFICACIÓN DE DATOS EN MILVUS ===")
print(f"Nombre de la colección: {collection.name}")
print(f"Número de entidades: {collection.num_entities}")

# Cargar la colección
collection.load()

# Consultar todos los datos
results = collection.query(expr="id >= 0", output_fields=["id", "metadata"], limit=10)

print(f"\n=== DATOS ENCONTRADOS: {len(results)} entidades ===")

for i, result in enumerate(results):
    print(f"\n--- Entidad {i+1} ---")
    print(f"ID: {result['id']}")
    
    # Mostrar metadatos de forma legible
    metadata = result['metadata']
    print(f"Título: {metadata.get('title', 'N/A')}")
    print(f"Idioma: {metadata.get('language', {}).get('lang', 'N/A')}")
    print(f"Tokens: {metadata.get('token_count', 'N/A')}")
    print(f"Keywords: {metadata.get('keywords', [])[:3]}...")  # Solo las primeras 3
    print(f"Resumen: {metadata.get('summary', 'N/A')[:100]}...")  # Solo los primeros 100 caracteres
    
    # Mostrar insights
    insights = metadata.get('insights', [])
    if insights:
        print(f"Insights: {len(insights)} encontrados")
        for j, insight in enumerate(insights[:2]):  # Solo los primeros 2
            print(f"  {j+1}. {insight[:80]}...")
    
    # Mostrar estructura
    structure = metadata.get('structure', [])
    if structure:
        print(f"Estructura: {len(structure)} secciones")
        for j, section in enumerate(structure[:2]):  # Solo las primeras 2
            print(f"  {j+1}. {section.get('section_title', 'N/A')}")

print("\n=== VERIFICACIÓN COMPLETADA ===")
print("Si ves datos aquí pero no en Attu, el problema es de visualización en Attu.")
print("Prueba a:")
print("1. Recargar la página de Attu")
print("2. Verificar que no hay filtros activos")
print("3. Hacer clic en el campo 'metadata' para expandirlo")
print("4. Reiniciar Attu si es necesario")

=== VERIFICACIÓN DE DATOS EN MILVUS ===
Nombre de la colección: documentos_legales_v2
Número de entidades: 0

=== DATOS ENCONTRADOS: 1 entidades ===

--- Entidad 1 ---
ID: 458955001465092393
Título: CONTRATO_COMPRAVENTA_INMUEBLE
Idioma: es
Tokens: 1005
Keywords: ['CONTRATO DE COMPRAVENTA', 'INMUEBLE', 'VENDEDOR']...
Resumen: El documento titulado 'CONTRATO DE COMPRAVENTA DE INMUEBLE' detalla un acuerdo realizado en Málaga e...
Insights: 5 encontrados
  1. El contrato establece que el precio total de la compraventa es de 250,000 euros,...
  2. El VENDEDOR garantiza que el inmueble se encuentra libre de cargas y gravámenes,...
Estructura: 4 secciones
  1. REUNIDOS
  2. INTERVIENEN

=== VERIFICACIÓN COMPLETADA ===
Si ves datos aquí pero no en Attu, el problema es de visualización en Attu.
Prueba a:
1. Recargar la página de Attu
2. Verificar que no hay filtros activos
3. Hacer clic en el campo 'metadata' para expandirlo
4. Reiniciar Attu si es necesario


In [16]:
from pymilvus import connections, Collection
import numpy as np

# Conectar a Milvus
connections.connect("default", host="localhost", port="19530")
collection = Collection("documentos_legales_v2")

print("=== PRUEBA DE BÚSQUEDA VECTORIAL ===")

# Cargar la colección
collection.load()

# Crear un vector de búsqueda (ejemplo: embedding de "contrato compraventa")
# Usando un vector aleatorio de 384 dimensiones como ejemplo
search_vector = np.random.rand(384).tolist()

# Realizar búsqueda vectorial
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10}
}

results = collection.search(
    data=[search_vector],
    anns_field="embedding", 
    param=search_params,
    limit=5,
    output_fields=["id", "metadata"]
)

print(f"Búsqueda completada. Resultados encontrados: {len(results[0])}")

for i, result in enumerate(results[0]):
    print(f"\n--- Resultado {i+1} ---")
    print(f"ID: {result.id}")
    print(f"Distancia: {result.distance}")
    
    metadata = result.entity.get('metadata')
    if metadata:
        print(f"Título: {metadata.get('title', 'N/A')}")
        print(f"Keywords: {metadata.get('keywords', [])[:3]}...")

print("\n=== PRUEBA COMPLETADA ===")
print("Si ves resultados aquí, el pipeline está funcionando perfectamente.")
print("El problema es solo de visualización en Attu.")

=== PRUEBA DE BÚSQUEDA VECTORIAL ===
Búsqueda completada. Resultados encontrados: 1

--- Resultado 1 ---
ID: 458955001465092393
Distancia: 134.39529418945312
Título: CONTRATO_COMPRAVENTA_INMUEBLE
Keywords: ['CONTRATO DE COMPRAVENTA', 'INMUEBLE', 'VENDEDOR']...

=== PRUEBA COMPLETADA ===
Si ves resultados aquí, el pipeline está funcionando perfectamente.
El problema es solo de visualización en Attu.


In [12]:
from pymilvus import connections, Collection

# Conectar a Milvus
connections.connect("default", host="localhost", port="19530")
collection = Collection("documentos_legales_v2")

print("=== VERIFICACIÓN DEL SCHEMA ===")
print(f"Nombre de la colección: {collection.name}")

# Mostrar todos los campos del schema
print("\n=== CAMPOS DEL SCHEMA ===")
for field in collection.schema.fields:
    print(f"Campo: '{field.name}'")
    print(f"  - Tipo: {field.dtype}")
    print(f"  - Primary: {field.is_primary}")
    print(f"  - AutoID: {field.auto_id}")
    if hasattr(field, 'dim'):
        print(f"  - Dimensión: {field.dim}")
    print()

# Verificar si hay índices
print("=== ÍNDICES ===")
try:
    indexes = collection.indexes
    for index in indexes:
        print(f"Índice: {index.params}")
except Exception as e:
    print(f"Error al obtener índices: {e}")

# Verificar el número de entidades
print(f"\n=== ESTADÍSTICAS ===")
print(f"Número de entidades: {collection.num_entities}")

# Intentar una consulta simple para ver qué campos están disponibles
print("\n=== PRUEBA DE CONSULTA ===")
try:
    results = collection.query(expr="id >= 0", output_fields=["*"], limit=1)
    if results:
        print("Campos disponibles en los resultados:")
        for key in results[0].keys():
            print(f"  - {key}")
    else:
        print("No hay resultados")
except Exception as e:
    print(f"Error en consulta: {e}")

print("\n=== FIN ===")


=== VERIFICACIÓN DEL SCHEMA ===
Nombre de la colección: documentos_legales_v2

=== CAMPOS DEL SCHEMA ===
Campo: 'id'
  - Tipo: 5
  - Primary: True
  - AutoID: True
  - Dimensión: None

Campo: 'embedding'
  - Tipo: 101
  - Primary: False
  - AutoID: False
  - Dimensión: 384

Campo: 'metadata'
  - Tipo: 23
  - Primary: False
  - AutoID: False
  - Dimensión: None

=== ÍNDICES ===
Índice: {'metric_type': 'L2', 'index_type': 'IVF_FLAT', 'params': {'nlist': 128}}

=== ESTADÍSTICAS ===
Número de entidades: 0

=== PRUEBA DE CONSULTA ===
Campos disponibles en los resultados:
  - id
  - embedding
  - metadata

=== FIN ===
